# Lab 5: Land Surface Temperature using CORINE-based Emissivity

## 🎯 Objectives
In this exercise, you will:
- Select a cloud-free Landsat 8 images from 2013 and 2023 (or different if you're sure that you'll spot a difference in land cover)
- Calculate Brightness Temperature (TB) from Band 10.
- Load CORINE Land Cover data and assign emissivity values to each land cover class.
- Use the Planck-based formula to calculate Land Surface Temperature (LST).
- Visualize and interpret the results.

In [48]:
import geemap
import ee
import numpy as np

ee.Authenticate()
ee.Initialize(project='rsia-lab')

## Step 1: Define Area of Interest (AOI)
- Use coordinates around Reduta street in Kraków.
- Create a polygon or rectangle using `ee.Geometry.Polygon`.

In [49]:
# Define AOI here
aoi = ee.Geometry.Polygon(    [
        [19.972, 50.092], 
        [19.972, 50.102],  
        [19.982, 50.102],  
        [19.982, 50.092],  
        [19.972, 50.092]  
    ])

## Step 2: Load Landsat 8 imagery for the dates you've picked
- Filter for low cloud cover (< 20%)
- Select Band 10 and convert to TB using: `TB = ST_B10 * 0.00341802 + 149.0`

In [50]:
# Load image and calculate TB

image_2013 = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2') \
    .filterBounds(aoi) \
    .filterDate('2013-01-01', '2013-12-31') \
    .sort('CLOUD_COVER') \
    .filterMetadata('CLOUD_COVER', 'less_than', 20) \
    .first()

image_2023 = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2') \
    .filterBounds(aoi) \
    .filterDate('2023-01-01', '2023-12-31') \
    .sort('CLOUD_COVER') \
    .filterMetadata('CLOUD_COVER', 'less_than', 20) \
    .first()

print('Image 2013:', image_2013.getInfo()['properties']['CLOUD_COVER'])
print('Image 2023:', image_2023.getInfo()['properties']['CLOUD_COVER'])

TB_2013 = image_2013.select('ST_B10').multiply(0.00341802).add(149)
TB_2023 = image_2023.select('ST_B10').multiply(0.00341802).add(149)


Image 2013: 0.02
Image 2023: 0.93


## Step 3: Load CORINE Land Cover data
- Use dataset `COPERNICUS/CORINE/V20/100m/2018`
- Clip it to your AOI

In [51]:
# Load CORINE
corine = ee.Image('COPERNICUS/CORINE/V20/100m/2018')

# Clip the CORINE dataset to the AOI
corine_clipped = corine.clip(aoi)

## Step 4: Assign emissivity to CORINE classes
- Use a dictionary for classes
- Use `remap()` and optionally a default value

In [52]:
# Create emissivity image

emissivity_dict = {
    111: 0.92,  # Continuous urban fabric
    112: 0.92,  # Discontinuous urban fabric
    121: 0.91,  # Industrial or commercial units
    211: 0.96,  # Non-irrigated arable land
    311: 0.98,  # Forests
    412: 0.97,  # Peat bogs
    324: 0.96,  # Transitional woodland-shrub
    231: 0.97   # Pastures
}

cornie_classes = list(emissivity_dict.keys())
emissivity_values = list(emissivity_dict.values())

emissivity_image = corine_clipped.remap(cornie_classes, emissivity_values, defaultValue = np.mean(emissivity_values))


## Step 5: Calculate LST using the formula:
$$
LST = \frac{T_B}{1 + \left( \frac{\lambda \cdot T_B}{c_2} \right) \cdot \ln(\varepsilon)}
$$
- λ = 10.8 µm
- c₂ = 14388 µm·K



In [53]:
# Calculate LST
LST_lambda = 10.8e-6
LST_C_2 = 1.4388e-2
log_emissivity = emissivity_image.log()

LTS_denominator_2013 = ((TB_2013.multiply(LST_lambda / LST_C_2)).multiply(log_emissivity)).add(1) 
LST_2013 = TB_2013.divide(LTS_denominator_2013)

LTS_denominator_2023 = ((TB_2023.multiply(LST_lambda / LST_C_2)).multiply(log_emissivity)).add(1) 
LST_2023 = TB_2023.divide(LTS_denominator_2023)

LTS_diff = LST_2023.subtract(LST_2013)


## Step 6: Visualize the LST result
- Use palette: `['blue', 'yellow', 'red']`
- Suggested range: `min=290`, `max=325`

In [54]:
# Visualize LST
Map = geemap.Map()
Map.centerObject(aoi, 14)

Map.addLayer(LST_2013, 
            {
                'min': 290, 
                'max': 325, 
                'palette': ['blue', 'yellow', 'red'] 
                }, 
                'LST 2013')

Map.addLayer(LST_2023, 
            {
            'min': 290, 
            'max': 325, 
            'palette': ['blue', 'yellow', 'red'] 
            }, \
            'LST 2023')

Map.addLayer(LTS_diff, 
            {
            'min': -5, 
            'max': 5, 
            'palette': ['blue', 'yellow', 'red'] 
            }, \
            'LTS diff')

Map

Map(center=[50.096999933442696, 19.97700000000005], controls=(WidgetControl(options=['position', 'transparent_…

**Analysis**

The results show that land temperatures in 2013 were higher than in 2023. This temperature difference is nicely shown on the *LTS diff* map, where most areas are covered in blue, indicating a negative difference between the two years. Additionally, LTS 2013 and LTS 2023 indicate that temperatures on streets and built-up area are higher than in green once.

## Step 7: (Optional) Analyze statistics by land cover class

In [55]:
# Optionally compute zonal statistics



## Step 8: (Optional - Easter Egg :)) Generate your own Land Cover Classification using TerraTorch and foundation models*

Based on the example/tutorial: https://aiforgood.itu.int/event/workshop-earth-observation-foundation-models-with-prithvi-eo-2-0-and-terratorch/

*to earn 5.0 grade that will make a great impact on your final grade